__This framework prepared by Hari Thapliyal (PGDS - IIITB, Cohort10) hari.prasad@vedavit-ps.com__<br>
__Any suggestion to improve are most welcome__

All the steps mentioned are suggestive. Based on the datasize, number of features, 
type of columns, goal of ML project (regression, classification, clustering, timeseries-regression) you can opt-out some steps or add some more

<font color=red size=6>__Loading All Needed Libraries__</font>

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')
import gc # for deleting unused variables

#Number Manupulation & Plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Model Selection Libraries
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve

#Preprocessing Libraries
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

#Liner Model Libraries
#from sklearn import linear_model
#from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import Ridge
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import ElasticNet

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance


#pipeline libraries
#from sklearn.pipeline import Pipeline
#from sklearn.pipeline import make_pipeline

#Metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, roc_curve, auc

import sklearn
import sklearn.decomposition

from sklearn.svm import SVC
#from sklearn.feature_selection import RFE

#import statsmodels.api as sm
#from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#folder= "/content/drive/My Drive/Colab Notebooks/Datasets-Kaggle/"
folder=""
file1 = folder + "prudential_train_clean_data.csv"
file2 = folder + "prudential_all_fldmap.csv"

<font color=red size=6>__Define Objective__</font>

<font color=red size=6>__Understanding Dataset__</font>

__Identify Fields which has more than 80% null values__ <br>
_Drop those columns or write down your approach here how will you impute those_

__Prepare list of different type of columns for processing them in group<br>
Record Identified, typical "Id"<br>
Target Variable, typical "Response", "Target", "Class" etc, this is also called y, which you have to predict<br>
Categorical Nominal Columns<br>
Categorical Oridinal Columns<br>
Date/time columns<br>
Numerical Columns<br>
Id & Target Variable never scaled<br>
Oridnal columns has hotencoding<br>
Nomial Columns has dummy columns<br>__
_ONLY Numerical columns need to be scaled<br>_

In [ ]:
target="Response"

In [ ]:
df = pd.read_csv("prudential_train.csv")

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
print('df uses {0} MB'.format(df.memory_usage().sum()/1024**2))
#print(df.memory_usage())
df.info()

In [ ]:
df.select_dtypes("int64").max().sort_values()[::-1]
tmp_cols =dict(df.select_dtypes("int64").max().sort_values()[::-1])

for k,v in tmp_cols.items():
    if v>32000:
        df[k] = df[k].astype("uint32")
    else:
        df[k] = df[k].astype("uint16")

#df.select_dtypes("float64").max().sort_values()[::-1]
tmp_cols = df.select_dtypes("float64").columns

df[tmp_cols] = df[tmp_cols].astype("float32")

#df.select_dtypes("object").max().sort_values()[::-1]

df.info()

In [ ]:
#Check Null Value Columns
(df.isnull().sum()/len(df)*100).sort_values()[::-1].head(15)

#Identify Fields which has more than 90% null values
f= list((df.isnull().sum() /len(df)*100) >45)
cols_45perNullVal = df.iloc[:,f].columns

print ("Dropping {} fields: {}".format( len(cols_45perNullVal), cols_45perNullVal))
df.drop(columns=cols_45perNullVal, inplace=True)

In [ ]:
def getBinary_flds(df):
    bin_flds = []
    for col in df.columns:
        if len(df[col].unique())==2:
            bin_flds.append(col)
    return bin_flds

In [ ]:
#Create a list of Categorical Variables. This step is useful when there are many fields in dataset
#If they are numeric in nature, convert into Object
#In final dataset all categorical variable (ordinal, nominal) will be object type
def getCat_flds(df):
    categories = min(30, int(len(df)*.5))
    cat_flds = []
    for col in df.columns:
        if len(df[col].unique())<categories:
            cat_flds.append(col)
            print (col, df[col].unique())
    return cat_flds

In [ ]:
cat_flds = getCat_flds(df)

In [ ]:
#None of these categorical field are ordinal field. 
#If there is any then we need to list them separately in the below column identification step
print ("There are %d categorical fields in dataset"%(len(cat_flds)) )

In [ ]:
############################All Columns of Different Types
##All Numerial Fields in Dataset
cols_all_num = list(set(list(df.columns)).difference(set(cat_flds)))

##All Categorical Fields in Dataset
cols_all_cat = cat_flds

#if target variable is categorical then remove from cols_all_cat list and else remove from cols_all_num
try:
    cols_all_num.remove(target)
except: pass

try:
    cols_all_cat.remove(target)
except: pass
    
##All Numerial Fields in Dataset which has Ordinal Variable
cols_all_ord  = []
#cols_all_ord  = ['OverallQual','OverallCond','HalfBath','FullBath','BsmtFullBath','BsmtHalfBath',
#                 'BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','MoSold',
#                 'YrSold','MSSubClass']

##All Numerial Fields in Dataset which has Ordinal Variable
cols_all_nom = list(set(cols_all_cat).difference(set(cols_all_ord)))


############################All Null Value Columns
#All Fields which has any Null values
fld_TF = list((df.isnull().sum() /len(df)*100) >0)
cols_with_NullVal_all = df.iloc[:,fld_TF].columns
#Numeric Fields which has any Null values
cols_with_NullVal_num= list(  df[cols_all_num].iloc[ :,list(df[cols_all_num].isna().sum()>0) ].columns )
#Categorical Fields which has any Null values
cols_with_NullVal_cat= list(  df[cols_all_cat].iloc[ :,list(df[cols_all_cat].isna().sum()>0) ].columns )
#Ordinal Fields which has Null Values
fld_TF = list((df[cols_all_ord].isnull().sum() /len(df)*100) >0)
cols_with_NullVal_ord = df.iloc[:,fld_TF].columns


###########################All Non-Null Value Columns
#All Fields which has any Null values
fld_TF = list((df.isnull().sum() /len(df)*100) ==0)
cols_with_NonNullVal_all =df.iloc[:,fld_TF].columns
#Numeric Fields which has any Null values
cols_with_NonNullVal_num= list(  df[cols_all_num].iloc[ :,list(df[cols_all_num].isna().sum()==0) ].columns )
#Categorical Fields which has any Null values
cols_with_NonNullVal_cat= list(  df[cols_all_cat].iloc[ :,list(df[cols_all_cat].isna().sum()==0) ].columns )
#Ordinal Fields which has Null Values
fld_TF = list((df[cols_all_ord].isnull().sum() /len(df)*100) ==0)
cols_with_NonNullVal_ord =df.iloc[:,fld_TF].columns

<font color=red size=6>__Check Null Value Fields & Existing Values in the Fields__</font><br>
__Null Value Categorical Nominal Columns<br>
Null Value Categorical Oridinal Columns<br>
Null Value Date/time columns<br>
Null Value Numerical Columns<br>
Null Value Id & Target Variable never scaled<br>
Null Value Oridnal columns has hotencoding<br>
Null Value Nomial Columns has dummy columns<br>__

<font color=red size=6>__Check Data Imbalance & Outliers__</font>

###### <font color=red size=4> Target Variable Data Imbalance</font>

<font color=red size=4> Categorical Variable Data Imbalance</fond>

<font color=red size=4> Oridinal Variable Data Imbalance</font>

<font color=red size=4> Numerical Variable Data Distribution</font>

<font color=red size=4> __Decide and take action on outliers at this stage__</font>

<font color=red size=6>__Imputing Null Values__</font>

<font color=red size=4> Impute ONLY numerical values using Iterative Imputer</font>

_Except Numerical columns restore the dataset as orginal_

<font color=red size=4> Impute ONLY  Ordinal values using KNN Imputer</font> <br>
_I have writen python function for this. This function convert existing non-null values into number, Impute null values using the number and returns a integer column. This function returns a column with new values, & map between old values and new number values_

_Except Numerical & Oridinal Value columns restore the dataset as orginal_

<font color=red size=4> Impute ONLY  Nominal values using KNN Imputer</font> <br>
_I have writen python function for this. This function convert existing non-null values into number, Impute null values using the number and returns a integer column. This function returns a column with new values, & map between old values and new number values_

<font color=blue size=5><center>__##########Data Cleaning Ends Here##########__<center></font>

<font color=red size=6>__Visualising Relationship between y & X__</font>

<font color=red size=4>Check y & Cateorical Variable Relationship</font> : _scatter plot is the best_

<font color=red size=4>Check y & Ordinal Variable Relationship</font> : _scatter plot is the best_

<font color=red size=4> Check y & Numeric Variable Relationship</font> : _scatter plot is the best_

<font color=red size=4> Check Degree of Correlation between y & Numeric Variable</font> : _Heatmap plot is the best_

<font color=red size=6>__Feature Engineering__</font>

__If target variable is not normally distributed. Then perform feature engineering.__

<font color=blue size=5><center>__####Preprocessing Work before Modelling Starts Here####__<center></font>

<font color=red size=3> Create Dummy Fields</font>

<font size=4 color=green>__PCA (Optional)__</font><br>
_Compress Variables in PCA, if number of variables are large_<br>
__Decide how many PCA features you need__

<font size=3 color=red>If PCA Done, then Visulise Relationship between Selected PCA Features & Target</font><br>

<font size=3 color=red>If PCA Done, then Feature Engineering on PCA Dataset may be required</font><br>

<font color=red size=3>Split Dataset in Train & Test</font>

<font color=red size=3>Split Dataset in Train & Test in X & y & id</font>

<font color=red size=3>Scale Numeric Fields</font><br>
__You can use StandardScalar<br>
fit_transform on train datset<br>
transform on test dataset__

 # <font color=red> <center> Enable Cell Below if reading cleaned data.</center></font>

In [ ]:
df= pd.read_csv(file1)
all_fldmap = pd.read_csv(file2)
all_fldmap = all_fldmap.drop(columns="Unnamed: 0")
df   = df.drop(columns="Unnamed: 0")

print('df uses {0} MB'.format(df.memory_usage().sum()/1024**2))
#print(df.memory_usage())
#df.info()

In [ ]:
bin_flds = getBinary_flds(df)

In [ ]:
df[bin_flds] = df[bin_flds].astype("uint8")

In [ ]:
print('df uses {0} MB'.format(df.memory_usage().sum()/1024**2))
#print(df.memory_usage())
df.info()

In [ ]:
#Dataset at this stage has dummy fields. All categorical columns converted into dummy fields.
#Identify columns which should be scale
colname=[]
colmin=[]
colmax=[]
for c in df.columns:
    colname.append(c)
    colmin.append( df[c].min())
    colmax.append( df[c].max())
df_minmax = pd.DataFrame( {"colname": colname, "colmin": colmin, "colmax": colmax})
#df_minmax.to_csv("minmax1.csv")

#if min is 0 and max=1 then it id dummy field. So we need not to scale
col_int = list(df_minmax[ ( (df_minmax.colmin==0) & (df_minmax.colmax==1) )].colname)

col_to_scale = df_minmax[~ ( (df_minmax.colmin==0) & (df_minmax.colmax==1) )]
col_to_scale = list(col_to_scale.colname)
col_to_scale

<font color=red size=5>__Split Dataset & Scale Numeric Fields__</font>

In [ ]:
#Split Dataset in X & y
Xcols = list(df.columns)
Xcols.remove(target)

X = df[Xcols]
y = df[target]

In [ ]:
# split X, y into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)
X_train_id = X_train.Id
X_test_id = X_test.Id

X_train = X_train.drop(columns="Id")
X_test = X_test.drop(columns="Id")

In [ ]:
print(y_train.mean())
print(y_test.mean())

In [ ]:
# scaling the numeric feature features in train and test
from sklearn.preprocessing import StandardScaler

col_to_scale.remove("Id")

try:
    col_to_scale.remove(target)
except Exception as e:
    print (e)
sc = StandardScaler()
X_train[col_to_scale] =pd.DataFrame( sc.fit_transform( X_train[col_to_scale] ), index=X_train.index )
X_test[col_to_scale]  =pd.DataFrame( sc.transform( X_test[col_to_scale] ) , index=X_test.index)

<font color=blue size=5><center>__##########Modelling Start Here##########__<center></font>

<font color=red size=8>__Start Modeling__</font><br>
- Here onwards you can build model either PCA dataset or Normal or Both<br>
- You need to try multiple models and check the performance of each<br>
- Either choose one or combine the results in one

<font color=blue size=5><center>__##########Only for Regression Problems##########__<center></font>

<font color=red size=5>__Linear Regression__</font><br>
_If number of variables are less then this is the best rather going for Lasso/Ridge_

<font color=red size=5>__Lasso Regression__</font><br>
_Build model using GridSearchCV_

In [ ]:
from sklearn.linear_model import Lasso

__Plot the score of GridSearchCV with alpha__

__Finalise the optimical value of alpha for Lasso & Redbuid the Model__

__Check Lasso Regression R Squre Score__

<font size=3 color=black>__Columns selected by Lasso May Have Multicolineaity So need to Address that__</font>

<font size=4 color=red>__Refining the model using linear regression, RFE and VIF__</font><br>
__Build linear regression model using statsmodel<br>
Use only those columns which are given by Lasso model<br>
Check statsmodel.summary() and drop columns of high p value<br>
Check VIF, drop the columns with high VIF<br>__
_All above steps are iterative in nature. At every step you remove multicolinearity and strengthen model.<br>
For removing features based on vif or pvalue and rebuilding the model I have writen python code_

__Finally Check Multi Colinearity using VIF__

<font size=4 color=red>__Build the final model using Lasso.__<font><br>
    Use only above selected features and earlier decided alpha

__Check R Square Score of the Final Lasso Model__

__Plot the Prediction Errors & their Randomness__

<font color=red size=5>__Ridge Regression__</font>

In [ ]:
from sklearn.linear_model import Ridge

__Ridge Regression Score__

<font size=5 color=red>__Elastic Regression__</font>

In [ ]:
from sklearn.linear_model import ElasticNet

__ElasticNet Regression Score__

<font size=5 color=red>__Decision Tree Regression__</font>

<font size=5 color=red>__K Nearest Neighbour Regression__</font>

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

<font size=5 color=red>__Random Forest__</font>

__RandomForest Score__

<font size=5 color=red>__XGBoosting__</font>

__XGBoosting Score__

<font color=blue size=5><center>__##########Only for Classification Problems Here##########__<center></font>

In [ ]:
def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

<font size=5 color=red>__Logistic Regression__</font>

In [ ]:
from sklearn.linear_model import LogisticRegression
model_logreg= LogisticRegression(class_weight="balanced")

#params = {"C": [.1,.2,.3,.4,.5,.6,.7,.8,.9], 'penalty':['l1','l2'], 'class_weight': ['balanced',{0:0.38, 1:0.62}]}
#model_logreg = LogisticRegression()
#grid_model_logreg = GridSearchCV(model_logreg, params_grid = params, scoring="recall", cv=5 )

model_logreg.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_logreg.predict(X_test), 
    y_score=model_logreg.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__Gaussian Naïve Bayes__</font>

In [ ]:
from sklearn.naive_bayes import GaussianNB
model_gnb = GaussianNB(class_weight="balanced")

#params = {"C": [.1,.2,.3,.4,.5,.6,.7,.8,.9], 'penalty':['l1','l2'], 'class_weight': ['balanced',{0:0.38, 1:0.62}]}
#model_logreg = LogisticRegression()
#grid_model_logreg = GridSearchCV(model_gnb, params_grid = params, scoring="recall", cv=5 )

model_gnb.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_gnb.predict(X_test), 
    y_score=model_gnb.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__Stochastic Gradient Descent Classifier__</font>

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss="modified_huber")

<font size=5 color=red>__K-Nearest Neighbors Classifier__</font>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=8)
model_knn.fit(X_train,y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_knn.predict(X_test), 
    y_score=model_knn.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__Hierarchical Tree__</font>

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

# plot a dendogram complete linkage
plt.figure(figsize=(15,5))
mergings = linkage(df_pca_country_scaled, method="complete", metric='euclidean')
dendrogram(mergings)
plt.show()

<font size=5 color=red>__SVM__</font>

In [ ]:
from sklearn.svm import SVC
model_svm = SVC(kernel="linear", C=.025, random_state=101)

#folds = 3
#grid_search_svm = GridSearchCV(model_svm, 
#                               cv = folds,
#                               param_grid=param_grid, 
#                               scoring = 'roc_auc', 
#                               return_train_score=True,                         
#                               verbose = 1)

#grid_search_svm.fit(X_train,  y_train)

model_svm.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_svm.predict(X_test), 
    y_score=model_svm.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__Decision Tree Classifier__</font>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_dtree = DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, min_impurity_split=10)

#folds = 3
#grid_search_dtree = GridSearchCV(model_dtree, 
#                               cv = folds,
#                               param_grid=param_grid, 
#                               scoring = 'roc_auc', 
#                               return_train_score=True,                         
#                               verbose = 1)

#grid_search_dtree.fit(X_train,  y_train)

model_dtree.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_dtree.predict(X_test), 
    y_score=model_dtree.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__Randomforest Classifier__</font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
param_grid = {"base_estimator__max_depth" : [2, 5],
              "n_estimators": [200, 400, 600]
              }

model_rf = RandomForestClassifier(max_features=10, min_samples_leaf=50, min_samples_split=50,
                            random_state=100, n_estimators=100)

#folds = 3
#grid_search_rf = GridSearchCV(model_rf, 
#                               cv = folds,
#                               param_grid=param_grid, 
#                               scoring = 'roc_auc', 
#                               return_train_score=True,                         
#                               verbose = 1)

#grid_search_rf.fit(X_train,  y_train)

model_rf.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_rf.predict(X_test), 
    y_score=model_rf.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__AdaBoost__</font>

In [ ]:
param_grid = {"base_estimator__max_depth" : [2, 5],
              "n_estimators": [200, 400, 600]
              }
              
    
# base estimator
tree = DecisionTreeClassifier(max_depth=2)

# adaboost with the tree as base estimator
model_adaboost = AdaBoostClassifier(base_estimator=tree, n_estimators=600, learning_rate=1.5,algorithm="SAMME")

#folds = 3
#grid_search_adb = GridSearchCV(model_adaboost, 
#                               cv = folds,
#                               param_grid=param_grid, 
#                               scoring = 'roc_auc', 
#                               return_train_score=True,                         
#                               verbose = 1)

#grid_search_adb.fit(X_train,  y_train)
model_adaboost.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_adaboost.predict(X_test), 
    y_score=model_adaboost.predict_proba(X_test))

print(report_with_auc)

In [ ]:
# cv results
cv_results = pd.DataFrame(grid_search_adb.cv_results_)
cv_results

In [ ]:
# plotting AUC with hyperparameter combinations

plt.figure(figsize=(16,6))
for n, depth in enumerate(param_grid['base_estimator__max_depth']):
    

    # subplot 1/n
    plt.subplot(1,3, n+1)
    depth_df = cv_results[cv_results['param_base_estimator__max_depth']==depth]

    plt.plot(depth_df["param_n_estimators"], depth_df["mean_test_score"])
    plt.plot(depth_df["param_n_estimators"], depth_df["mean_train_score"])
    plt.xlabel('n_estimators')
    plt.ylabel('AUC')
    plt.title("max_depth={0}".format(depth))
    plt.ylim([0.60, 1])
    plt.legend(['test score', 'train score'], loc='upper left')
    plt.xscale('log')

<font size=5 color=red>__Gradient Boost__</font>

In [ ]:
# parameter grid
param_grid = {"learning_rate": [0.2, 0.6, 0.9],
              "subsample": [0.3, 0.6, 0.9],
              "num_class":[1,2,3,4,5,6,7,8]
             }
# adaboost with the tree as base estimator
model_gbc = GradientBoostingClassifier(max_depth=2, n_estimators=200)

# run grid search
#folds = 3
#grid_search_gbc = GridSearchCV(model_gbc, 
#                               cv = folds,
#                               param_grid=param_grid, 
#                               scoring = 'roc_auc', 
#                               return_train_score=True,                         
#                               verbose = 1)

#grid_search_gbc.fit(X_train, y_train)
model_gbc.fit(X_train, y_train)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=gbc.predict(X_test), 
    y_score=gbc.predict_proba(X_test))

print(report_with_auc)

<font size=5 color=red>__XGBoost__</font>

In [ ]:
# hyperparameter tuning with XGBoost

# creating a KFold object 
folds = 3

# specify range of hyperparameters
param_grid = {'learning_rate': [0.2, 0.6], 
             'subsample': [0.3, 0.6, 0.9]}          


# specify model
model_xgbc = XGBClassifier(max_depth=2, objective = 'multi:softmax', n_estimators=200, num_class=8)

# set up GridSearchCV()
#model_cv = GridSearchCV(estimator = model_xgbc, 
#                        param_grid = param_grid, 
#                        scoring= 'roc_auc', 
#                        cv = folds, 
#                        verbose = 1,
#                        return_train_score=True) 
#model_cv.fit(X_train, y_train)
model_xgbc.fit(X_train,y_train)

In [ ]:
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=model_xgbc.predict(X_test))
print(sk_report)

In [ ]:
report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model_xgbc.predict(X_test), 
    y_score=model_xgbc.predict_proba(X_test))

print(report_with_auc)

<font color=blue size=5><center>__##########Only for Clustering Problems##########__<center></font>

<font size=5 color=red>__KNN__</font>

In [ ]:
#Let us see 10 clusters and size of of each clsuter with kmean
kmeans = KMeans(n_clusters=10, max_iter=50, random_state=100)
kmeans.fit(df)

cluster_labels_km = kmeans.labels_
#Using K means we get following number of countries in each each cluster
pd.Series(cluster_labels_km).value_counts()

## Hopkins Statistics:
The Hopkins statistic, is a statistic which gives a value which indicates the cluster tendency, in other words: how well the data can be clustered.

- If the value is between {0.01, ...,0.3}, the data is regularly spaced.
- If the value is around 0.5, it is random.
- If the value is between {0.7, ..., 0.99}, it has a high tendency to cluster.

In [ ]:
def hopkins(X):
    d = X.shape[1]

    n = len(X) # rows
    m = int(0.1 * n) 
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),
                                            np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0

In [ ]:
#Use the Hopkins Statistic function by passing the above dataframe as a paramter
hopkins(df)

<font size=5 color=red>__Hierarchical Tree__</font>

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

# plot a dendogram complete linkage
plt.figure(figsize=(15,5))
mergings = linkage(df, method="complete", metric='euclidean')
dendrogram(mergings)
plt.show()

##### Silhouette Analysis
$$\text{silhouette score}=\frac{p-q}{max(p,q)}$$<br>
$p$ is the mean distance to the points in the nearest cluster that the data point is not a part of
$q$ is the mean intra-cluster distance to all the points in its own cluster.<br>

* The value of the silhouette score range lies between -1 to 1. 
* A score closer to 1 indicates that the data point is very similar to other data points in the cluster, 
* A score closer to -1 indicates that the data point is not similar to the data points in its cluster.

In [ ]:
# Cluster Analysis using silhouette score & Elbow curve
sse_=[]
ssd =[]
for num_clusters in range(2,11):
    
    # intialise kmeans
    kmeans1 = KMeans(n_clusters=num_clusters, max_iter=50, random_state=100).fit(df_pca_country_scaled)
    
    # silhouette score
    silhouette_avg = silhouette_score(df_pca_country_scaled, kmeans1.labels_)
    
    sse_.append([num_clusters, silhouette_avg])
    
    #score for elbow curve
    ssd.append(kmeans1.inertia_)

In [ ]:
# silhouette analysis
# From this analysis it looks ideal number of cluster should be 5
for j in sse_:
    print("For n_clusters={0}, the silhouette score is {1}".format(j[0], j[1]))
    
plt.plot(pd.DataFrame(sse_)[0], pd.DataFrame(sse_)[1]);

In [ ]:
#Elbow Curve Method
plt.plot(ssd)
plt.show()

In [ ]:
#if we cut the dendogram at 5 that will gives has 10 clusters.

cluster_labels_hc = cut_tree(mergings, n_clusters=10).reshape(-1, )
cluster_labels_hc

#Number of contries in each cluster
pd.Series(cluster_labels_hc).value_counts()

<font color=blue size=5><center>__##########Model Evaluation Ends Here###########__<center></font>

<font size=6 color=red>__Final Model__</font>
- Evaluate all the possible model
- Take the best possible hyperparamter and their values
- Build the final model here
- If required combine the results of all models and unify models
- Finally predict with final model

<font color=blue size=5><center>__##########Model Selection Ends Here##########__<center></font>

<font size=6 color=red>__Final List of Feature For Management__</font>

_During Dummy Encoding & Hotencoding original value of the categorical field is converted into numbers<br>
We need to convert these number into original values, so that management can understand them_

<font size=4 color=red>__Final Prediction Using All the Models__</font>

<font size=4 color=red>__Finally Visualise the Relationship between SalesPrice & Features__</font>

<font size=4 color=red>__Final List of Features & Coefficients For Management__</font>